<a href="https://colab.research.google.com/github/cyberone1812/CMUDeepLearning/blob/main/clean_dele_week9_cmu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Week 9 CMU Deep Learning

- [Example 1](https://sebastianraschka.com/blog/2023/self-attention-from-scratch.html)

In [ ]:
pip install datasets

In [ ]:
import torch
import torch.nn as nn

In [ ]:
#attention
sentence = "Alice went down the rabbit hole."

In [ ]:
sentence.split()

['Alice', 'went', 'down', 'the', 'rabbit', 'hole.']

In [ ]:
dc = {w:i for i, w in enumerate(sorted(sentence.split()))}

In [ ]:
dc

{'Alice': 0, 'down': 1, 'hole.': 2, 'rabbit': 3, 'the': 4, 'went': 5}

In [ ]:
dc['Alice']

0

In [ ]:
[dc[i] for i in sentence.split()]

[0, 5, 1, 4, 3, 2]

In [ ]:
num_sentence = torch.tensor([dc[i] for i in sentence.split()])

In [ ]:
num_sentence

tensor([0, 5, 1, 4, 3, 2])

In [ ]:
embedding_layer = nn.Embedding(len(num_sentence), 5)

In [ ]:
embedding_layer(num_sentence)

tensor([[ 0.6941,  0.2592,  1.4015, -0.8204, -0.7528],
        [-0.2364,  0.6361, -1.5011, -0.6235, -0.4114],
        [-0.3128,  0.4857,  0.3493, -0.7419, -1.3527],
        [ 0.4443, -0.4584,  1.7657,  0.7634,  0.4649],
        [-0.6630, -0.3596, -1.5551, -0.6986,  1.7498],
        [-0.6778, -0.1350, -2.2429,  1.2849,  0.2349]],
       grad_fn=<EmbeddingBackward0>)

In [ ]:
embedded_sentence = embedding_layer(num_sentence)

In [ ]:
d_q, d_k, d_v = 4, 4, 6

In [ ]:
d = 5 #num columns in embeddings

In [ ]:
Wq = nn.Parameter(torch.rand(d_q, d))
Wk = nn.Parameter(torch.rand(d_k, d))
Wv = nn.Parameter(torch.rand(d_v, d))

In [ ]:
x_1 = embedded_sentence[0]

In [ ]:
x_1

tensor([ 0.6941,  0.2592,  1.4015, -0.8204, -0.7528],
       grad_fn=<SelectBackward0>)

In [ ]:
Wq.matmul(x_1)

tensor([0.3621, 0.6564, 0.5727, 0.7489], grad_fn=<MvBackward0>)

In [ ]:
keys = Wk.matmul(embedded_sentence.T).T
values = Wv.matmul(embedded_sentence.T).T

In [ ]:
keys.shape

torch.Size([6, 4])

In [ ]:
values.shape

torch.Size([6, 6])

In [ ]:
#apply to one input vector
q1 = Wq.matmul(x_1)
k1 = Wk.matmul(x_1)
v1 = Wv.matmul(x_1)

In [ ]:
#apply to all inputs
omega_1 = q1.matmul(keys.T)

In [ ]:
omega_1

tensor([ 1.0153, -2.4124, -1.9498,  3.4301, -2.0831, -1.6585],
       grad_fn=<SqueezeBackward4>)

Attention scores:

$$\alpha_{1, i} = \text{softmax}\big(\frac{w_{1, i}}{\sqrt{d_k}})$$

In [ ]:
softmax = nn.Softmax(dim = 0)

In [ ]:
attention_weights = softmax(omega_1/torch.sqrt(torch.tensor(d_k)))

In [ ]:
attention_weights.shape

torch.Size([6])

In [ ]:
context_vector = attention_weights.matmul(values)

In [ ]:
context_vector.shape

torch.Size([6])

#### Huggingface

In [ ]:
from transformers import pipeline

In [ ]:
#transformer
speech_to_text = pipeline(task = "automatic-speech-recognition")

No model was supplied, defaulted to facebook/wav2vec2-base-960h and revision 55bb623 (https://huggingface.co/facebook/wav2vec2-base-960h).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You sho

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

In [ ]:
data = 'https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac'

In [ ]:
#pretrained
speech_to_text(data)

{'text': 'I HAVE A DREAM BUT ONE DAY THIS NATION WILL RISE UP LIVE UP THE TRUE MEANING OF ITS TREES'}

In [ ]:
from IPython.display import Audio

In [ ]:
Audio(data)

In [ ]:
#huggingface or spacy
data = 'https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/1.flac'

In [ ]:
speech_to_text(data)

{'text': 'HE HOPED THERE WOULD BE STEW FOR DINNER TURNIPS AND CARROTS AND BRUISED POTATOES AND FAT MUTTON PIECES TO BE LADLED OUT IN THICK PEPPERED FLOWER FAT AND SAUCE'}

In [ ]:
Audio(data)

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset(
    "hf-internal-testing/librispeech_asr_dummy", "clean", split="validation"
)
dataset

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for hf-internal-testing/librispeech_asr_dummy contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hf-internal-testing/librispeech_asr_dummy
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating validation split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 73
})

In [ ]:
dataset[0]['audio']

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/dfbece23564f422bc5794f3090902cd16d52d86767b746125ebc2ff3ea5f89ef/dev_clean/1272/128104/1272-128104-0000.flac',
 'array': array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00042725, 0.00057983,
        0.0010376 ]),
 'sampling_rate': 16000}

In [ ]:
Audio(dataset[0]['audio']['array'], rate = dataset[0]['audio']['sampling_rate'])

In [ ]:
pipe2 = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-100h")

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-100h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.mask_time_emb_vector']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-100h and are newly initialized: ['wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametriz

tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/358 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

In [ ]:
speech_to_text(dataset[1]['audio'])

{'text': "NOR IS MISTER QUILTER'S MANNER LESS INTERESTING THAN HIS MATTER"}

In [ ]:
pipe2(dataset[1]['audio'])

{'text': "NOR IS MISTER CILTER'S MANNER LESS INTERESTING THAN HIS MATTER"}